# python-pso

## Particle Swarm Optimizer (PSO)

This is a common implementation of the PSO, only with some extensions. Meaning the algorithm runs a set number of iterations or if the sd of the swarm is less than a predefined value (this is relatively arbitrary, in this case the square root of the product(self.n * self.dims), which seemed to be a reasonable idea).

Every iteration consists mainly of two steps:

- evaluation the field at a given place for every particle
- and updating it's position according to their personal best solution and the best solution found in the swarm.

Every particle has a position in space and a vector defining it's velocity. As indicated, if a solution is better, it's updated. In this implementation you can set the relative time, when to start taking into account the global best solution. This can be done to give personal best solutions a greater impact to not miss global minima. For more complex problems you should consider using H-PSO or even PH-HPSO. Some more things I changed about the original algorithm:

- if the velocity is to low, speed up with a rand_speed_factor, correlating with the dimension of the problem.
- don't exceed a maximum speed.
- chaos for velocity
- limit the search space to the given problem (-n,n)

Some of them could be left out or extended or could be set in motion in a more elegant way. Some of the characteristics, e.g. the pair-wise deviation of the swarm I found in the paper or the script for swarm intelligence, others were found in an trial-and-error way. I just want to illustrate the main principles for myself and everybody interested.

The visualisation objects have to be initialized in a main method and passed as arguments. For more information about the classes look at 'my_visualisation.py'


![hoelder_table](pso-example.png)

See below for some example code.

Install dependencies and import the class PSO:

Set the number of particles and the maximum number of iterations you want to run. The number of dimensions is set to 2 to show the illustration. I implemented some benchmark functions, which are evaluated and plotted.

Instantiate and instance of the PSO class and set the name of the function, which is to be evaluated (done by a dict-lookup).

Set the global update frame, i.e. the iterations, when the updates are made and run the optimizer.

## Step by step PSO Tutorial

Below you find a step by step tutorial of a simplified, yet more performant variant of the PSO algorithm. 
Asside from numpy no external dependency is needed. Make sure to import it.

In [3]:
import numpy as np
import sys

Since we want to evaluate a function, we need to define it before starting. I choose the Rastrigin function, which is a little more complex than x^2 but not too complicated. It can be replaced by any number of functions applicable to arrays. 

In [2]:
def rastrigin(row: np.ndarray):
    """
        apply rastrigin-function to matrix-row

    :param row:
    :return:
    """
    f_arr = np.frompyfunc(lambda d: d ** 2 - 10 * np.cos(np.pi * d) + 10, 1, 1)
    return np.sum(f_arr(row))

Next we need to set some constants. n depends on the function, the variable dims sets the dimensionality. 
Values c1 & c2 determined empirically (not by me) and are needed for the PSO algorithm.

In [3]:
n = 5.2
dims = 3
func = rastrigin

c1 = c2 = 1.494

num_particles = 10

Now we initialize the positions of the particles and their velocities. Together they represent our swarm and it's movement.

In [6]:
v = np.random.rand(num_particles, dims) * n +1
x = 2 * n * np.random.rand(num_particles, dims) - n
assert v.shape == x.shape

We will need to save the current best solutions of the particles as well as the global best solution. They will influence the positions or rather the velocities of the particles in our swarm.

In [5]:
best_solution = np.full((1, num_particles), sys.maxsize)
best_point = np.zeros_like(x)
best_global_point = np.zeros(dims)
best_global_solution = sys.maxsize

Now on to the good stuff. Since we defined a very nice function above, which makes it possible to apply a function to a row, we can apply it to all rows/ particles at once:

In [8]:
fx = np.apply_along_axis(func, axis=1, arr=x)

Now we determine the best solutions of all particles. If the current solution is not better than the best solution of a given particle, it is - of course - not changed at all. The index is also used to get the position of the best solution.

In [9]:
idx = fx < best_solution
best_solution[idx] = fx[idx[0]]
best_point[idx[0]] = x[idx[0], :]

Similary, we want to update the global best solution and points.

In [10]:
# update global best solutions
idx = np.argmin(best_solution)
if best_solution[:, idx][0] < best_global_solution:
    best_global_solution = best_solution[:,idx][0]
    best_global_point = x[idx, :]

After calculating the best solutions, we can update the velocities of the particles respective to the positions of their best solutions:
$$v = v + rand * c_1 * (best\_point -x)$$
The same is done with respect to position of the global best solution:
$$v = v + rand * c_2 * (global\_best\_point -x)$$

This will drag the particles into the area of the current best solution.

In [11]:
r = np.random.ranf(2)
v += r[0] * c1 * (best_point - x)
v += c2 * r[1] * (best_global_point - x)

Updating the particles is done by adding the velocites to the positions. In this implementation we don't want the particles to escape the search space, i.e. $-n, n$, thus we use np.clip.

In [12]:
x += v
x = np.clip(x, a_min=-n, a_max=n)
print(f"{best_global_solution}")

27


In [ ]:
r1 = np.random.ranf(dims)
# update highest velocity
if np.sum(np.abs(particle.v)) < np.sum(np.abs(max_vel[,])):
    max_vel = particle.v
    
# weird
if np.sum(r1) < np.sum(particle.v):
    v = r1 * v * max_vel ** rand_speed_factor
    # select random dimension
    ri = np.random.randint(dims)
    # constant factor to keep the chaos realistic
    x[ri] = r1[0] * ((2 * n) - n) * self.ws[i] * 0.45

Barely anthing happend? That's normal. The PSO algorithm is run iteratively, so lets add a for-loop an decreasing weights to decrease the influence of personal best solutions towards the end of the execution and wrap into a function.

In [1]:
def run_pso(num_particles, dims, iterations):


    n = 5.2
    func = rastrigin
    c1 = c2 = 1.494    
    
    v = np.random.rand(num_particles, dims) * n +1
    x = 2 * n * np.random.rand(num_particles, dims) - n
    assert v.shape == x.shape

    best_solution = np.full((1, num_particles), sys.maxsize)
    best_point = np.zeros_like(x)
    best_global_point = np.zeros(dims)
    best_global_solution = sys.maxsize
    ws = np.linspace(0.9, 0.4, iterations)

    for i in range(iterations):
        # apply function to all particles
        fx = np.apply_along_axis(func, axis=1, arr=x)

        # update best solutions of points
        idx = fx < best_solution
        best_solution[idx] = fx[idx[0]]
        best_point[idx[0]] = x[idx[0], :]

        # update global best solutions
        idx = np.argmin(best_solution)
        if best_solution[:,idx][0] < best_global_solution:
            best_global_solution = best_solution[:,idx][0]
            best_global_point = x[idx, :]

        # update velocity after formula given constants c1 & c2, some randomness,
        # and personal/ global best solutions.
        r = np.random.ranf(2)
        v = ws[i]*v + r[0] * c1 * (best_point - x)
        # you can start the global updating later, if you want.
        v += c2 * r[1] * (best_global_point - x)

        # update position
        x += v

        # keep all particles in range [-n, n]
        x = np.clip(x, a_min=-n, a_max=n)

        print(f"{best_global_solution}")

num_particles = 30
dims = 10
iterations = 100

#run_pso(num_particles, dims, iterations)

## Modifications
The Algorithm can be adjusted by adding even more chaos for both position and velocity. Let's first look at position, since it's easier to grasp. We first select a random dimension (you could also select multiple dimensions) and some particles, which we want to update. For this I chose 10% of the particles, which is arbitrary. After, the points are reset with a constant factor 0.45.

In [7]:
n = 5.
r_dim = np.random.randint(dims)
r_particles = np.random.choice(range(num_particles), num_particles//10)
x[r_particles, r_dim] = np.random.ranf() * ((2 * n) - n) * 0.45

Next, lets add some randomness to the velocities of the particles. We will need to keep track of the highest velocity and set a minimum velocity. I also added some code to add values once the particles move too slow. Notice that the particles a completely new velocity.

In [11]:
lowest_speed = 0.1
# don't fall asleep.
max_velocity = np.zeros(dims)
p_vel_abs = np.sum(np.abs(v), axis=1)
# update all particles, which have a speed which is below our threshold
v[p_vel_abs < lowest_speed, :] += np.random.ranf(dims)

# largest absolute velocity sum larger than max_vel
max_vel_sum = np.sum(np.abs(max_velocity))
# check if any particle is moving faster 
if np.any(max_vel_sum > p_vel_abs):
    # get index of particle with highest velocity
    _idx = np.argmax(np.sum((v[ max_vel_sum < p_vel_abs , :]), axis=1))
    # update max_velocity
    max_velocity = v[_idx, :]
# choose a random particle and update
r_particles = np.random.choice(range(num_particles), num_particles // 10)
# new velocities for selected particles
vals = np.random.ranf((len(r_particles), dims)) * max_velocity
v[r_particles, :] = vals

The code could be inserted directly into the main function. I chose to put into an inner function which let's me keep the code structure without moving everything into a class.

## Hierarchical Particle Swarm Optimizer (HPSO)

This is a basic implementation of the H-PSO-Algorithm. It's derived from PSO and has a tree-object. I figure this can be done way more elegantly, but then again, it's just for illustration. Most of the main-functions are called in recursive fashion. Since the tree-height shouldn't exceed 2 or max 3 levels, this can be implemented otherwise (not recursive).

The particles are hold in a tree-structure: the particle with the global best solution should be in the root, local best solutions are kept in the lower levels and leafs (although they are just 'normal' particles). Every parent has a direct impact on it's child. First every particle checks it's current solution and updates it's personal best if it's better. Then the particles swapp their places. This is done in the following way: if the solution of a child is better (i.e. lower) than it's parent, they change places. This is done top-bottom.

After, the particles change their position and speed according to their local and personal best solution. The weight changes with the level of the particle, ^HPSO and vHPSO are possible: vHPSO meaning the weight decreases towards the root. Unlike the PSO, I only added one addition, namely to stop if the diversity of the swarm is below a tolerance.

So far there is no implementation of PH-PSO. A first try can be found in update_hpso. The visualisation is done like in the PSO class.

The code execution works almost exactly as seen above. 

## Relevant literature:

James Kennedy and Russell Eberhart. Particle swarm optimization. In IEEE International Conference on Neural Networks (ICNN’95), volume 4, pages 1942–1947, Perth, Western Australia, November-December 1995. IEEE.

Trelea. The particle swarm optimization algorithm: Convergence analysis and parameter selection. IPL: Information Processing Letters, 85, 2003.

J. Kennedy and R. Mendes. Population structure and particle swarm performance. In David B. Fogel, Mohamed A. El-Sharkawi, Xin Yao, Garry Greenwood, Hitoshi Iba, Paul Marrow, and Mark Shackleton, ed- itors, Proceedings of the 2002 Congress on Evolutionary Computation CEC2002, pages 1671–1676. IEEE Press, 2002. ISBN 0-7803-7278-6.

P. N. Suganthan. Particle swarm optimizer with neighborhood opera- tor. In 1999 Congress on Evolutionary Computation, pages 1958–1962, Piscataway, NJ, 1999. IEEE Service Center. Trelea.